# Multi Environment JSON Pattern

This pattern allows you to manage configuration for multiple environments.

In modern software development, managing configurations across multiple environments has become a crucial aspect of maintaining robust applications. This document introduce a best practice to manage schema definition, data initialization, config deployment and config usage.

## Solution Overview

This solution ships the following best practices:

1. Utilize dataclasses to define the configuration schema, providing an object-oriented interface to access configuration values. This not only enables basic validation but also helps avoid typographical errors and offers auto-complete functionality within Integrated Development Environments (IDEs).
2. Separating the schema declaration from the data initialization enables a flexible strategy to load the config data from different sources in different environments. For example, you can load the config data from a local file in the development environment and retrieve the config data from AWS Parameter Store in the production environment.
3. Manage the source-of-truth of the config data in Git for non-sensitive data, enabling version control of the config data.
4. Leverage the inheritance hierarchy pattern to set global default values and be able to override them in specific environments.
5. Provide two options to deploy the config data to [AWS Parameter Store](https://docs.aws.amazon.com/systems-manager/latest/userguide/systems-manager-parameter-store.html) or [AWS S3](https://aws.amazon.com/s3/), with encryption-at-transit and encryption-at-rest enabled by default.
6. Isolate the deployment of configurations based on the environment, ensuring that each environment can only access its specific config data and cannot access the configuration of other environments. Additionally, maintain a versioned copy of the all-in-one configuration deployment as a backup for disaster recovery or compliance requirements.
7. Separate static config values from dynamic values, enabling developers to customize the logic for generating dynamic values.

To use this solution, you have to install the Python package [config_patterns](https://pypi.org/project/config-patterns/). It comes with the building block and the automation to implement your own multi environment config management system. Please continue to read to learn how to use it.

## Config Json File Format

In object-oriented programming, the inheritance hierarchy is a pattern where child objects inherit attributes and methods from parent objects. Similarly, in configuration management, the global configuration often acts as the default value, allowing for the possibility of overriding specific values in environment-specific configurations. Please take a look at the following example of config JSON file. We will explain it later.

In [1]:
import os
import json
from pathlib import Path
from rich import print as rprint

def jprint(data: dict):
    rprint(json.dumps(data, indent=4))

dir_here = Path(os.getcwd()).absolute()
path_config = dir_here.joinpath("config.json")

In [2]:
rprint(path_config.read_text())

# content of config.json
{
    # this config file support comments, you can put documentation in the config file
    "_shared": {
        "*.project_name": "my_project"
    },
    "dev": {
        "username": "dev.user"
    },
    "int": {
        "username": "int.user"
    },
    "prod": {
        "username": "prod.user"
    }
}

There's a meta field ``_shared`` in the root level of the config file. It is a powerful inheritance hierarchy mechanism to specify config values. The ``_shared`` field is a key value pairs of JSON path notation and it's value. For example:

In [3]:
from config_patterns.patterns.hierarchy import SHARED, apply_shared_value

data = {
    SHARED: {
        "*.name": "alice",
        "*.contact.email": "alice@email.com",
    },
    "dev": {
        "contact": {},
    },
    "prod": {
        "name": "bob",
        "contact": {
            "email": "bob@email.com"
        },
    },
}
print("before:")
jprint(data)
print("after:")
apply_shared_value(data)
jprint(data)

before:


{
    "_shared": {
        "*.name": "alice",
        "*.contact.email": "alice@email.com"
    },
    "dev": {
        "contact": {}
    },
    "prod": {
        "name": "bob",
        "contact": {
            "email": "bob@email.com"
        }
    }
}

after:


{
    "dev": {
        "contact": {
            "email": "alice@email.com"
        },
        "name": "alice"
    },
    "prod": {
        "name": "bob",
        "contact": {
            "email": "bob@email.com"
        }
    }
}

This mechanism is very powerful, it works with list of dict too.

In [4]:
data = {
    SHARED: {
        "*.databases.port": 5432
    },
    "dev": {
        "databases": [
            {"host": "db1.com"},
            {"host": "db2.com"},
        ],
    },
    "prod": {
        "databases": [
            {"host": "db3.com"},
            {"host": "db4.com"},
        ],
    },
}
print("before:")
jprint(data)
print("after:")
apply_shared_value(data)
jprint(data)

before:


{
    "_shared": {
        "*.databases.port": 5432
    },
    "dev": {
        "databases": [
            {
                "host": "db1.com"
            },
            {
                "host": "db2.com"
            }
        ]
    },
    "prod": {
        "databases": [
            {
                "host": "db3.com"
            },
            {
                "host": "db4.com"
            }
        ]
    }
}

after:


{
    "dev": {
        "databases": [
            {
                "host": "db1.com",
                "port": 5432
            },
            {
                "host": "db2.com",
                "port": 5432
            }
        ]
    },
    "prod": {
        "databases": [
            {
                "host": "db3.com",
                "port": 5432
            },
            {
                "host": "db4.com",
                "port": 5432
            }
        ]
    }
}

You can also specify default different value for different environment.

In [5]:
data = {
    SHARED: {
        "dev.databases.port": 5432,
        "prod.databases.port": 3306,
    },
    "dev": {
        "databases": [
            {"host": "db1.com"},
            {"host": "db2.com", "port": 0},
        ],
    },
    "prod": {
        "databases": [
            {"host": "db3.com"},
            {"host": "db4.com", "port": 1},
        ],
    },
}
print("before:")
jprint(data)
print("after:")
apply_shared_value(data)
jprint(data)

before:


{
    "_shared": {
        "dev.databases.port": 5432,
        "prod.databases.port": 3306
    },
    "dev": {
        "databases": [
            {
                "host": "db1.com"
            },
            {
                "host": "db2.com",
                "port": 0
            }
        ]
    },
    "prod": {
        "databases": [
            {
                "host": "db3.com"
            },
            {
                "host": "db4.com",
                "port": 1
            }
        ]
    }
}

after:


{
    "dev": {
        "databases": [
            {
                "host": "db1.com",
                "port": 5432
            },
            {
                "host": "db2.com",
                "port": 0
            }
        ]
    },
    "prod": {
        "databases": [
            {
                "host": "db3.com",
                "port": 3306
            },
            {
                "host": "db4.com",
                "port": 1
            }
        ]
    }
}

## Separate and Merge Non-Sensitive Config and Secret Config

You should not check-in sensitive config data like database password into Git. Usually, developer maintain a secret config file locally when they are doing development  on local laptop. This solution provides mechanism to automatically merge the secret config data into the config data. Please take a look at the following example:

In [6]:
from config_patterns.jsonutils import json_loads
print("config:")
config_data = json_loads(path_config.read_text())
rprint(config_data)
path_secret_config = dir_here.joinpath("secret_config.json")
print("secret config:")
secret_config_data = json_loads(path_secret_config.read_text())
rprint(secret_config_data)

config:


{
    '_shared': {'*.project_name': 'my_project'},
    'dev': {'username': 'dev.user'},
    'int': {'username': 'int.user'},
    'prod': {'username': 'prod.user'}
}

secret config:


{
    '_shared': {},
    'dev': {'password': 'dev.password'},
    'int': {'password': 'int.password'},
    'prod': {'password': 'prod.password'}
}

In [7]:
from config_patterns.patterns.merge_key_value import merge_key_value

apply_shared_value(config_data)
apply_shared_value(secret_config_data)
merged = merge_key_value(config_data, secret_config_data)
print("merged data:")
rprint(merged)

merged data:


{
    'dev': {'username': 'dev.user', 'project_name': 'my_project', 'password': 'dev.password'},
    'int': {'username': 'int.user', 'project_name': 'my_project', 'password': 'int.password'},
    'prod': {'username': 'prod.user', 'project_name': 'my_project', 'password': 'prod.password'}
}

## Sample Application Code

### Declare Your Config Schema

In software engineer best practice, declaration and the usage of a Data Model should be separated. Below is the ``config_define.py`` file that defines three things:

1. enumerate all environments you want to use in your project.
2. declare the per environment config data model.
3. subclass from the BaseConfig, this is your main config object.

In [8]:
# content of config_define.py
# -*- coding: utf-8 -*-

import typing as T
import dataclasses

from config_patterns.patterns.multi_env_json import (
    BaseEnvEnum,
    BaseEnv,
    BaseConfig,
)


class EnvEnum(BaseEnvEnum):
    dev = "dev" # development
    int = "int" # integration test
    prod = "prod" # production


@dataclasses.dataclass
class Env(BaseEnv):
    username: T.Optional[str] = dataclasses.field(default=None)
    password: T.Optional[str] = dataclasses.field(default=None)

    @classmethod
    def from_dict(cls, data: dict):
        return cls(**data)
    

@dataclasses.dataclass
class Config(BaseConfig):
    @property
    def dev(self) -> Env:
        return self.get_env(EnvEnum.dev)

    @property
    def int(self) -> Env:
        return self.get_env(EnvEnum.int)

    @property
    def prod(self) -> Env:
        return self.get_env(EnvEnum.prod)

    @classmethod
    def get_current_env(cls) -> str:
        return EnvEnum.dev.value

    @property
    def env(self) -> Env:
        return self.get_env(self.get_current_env())

### Read From Local File

As the project admin, you need to decide what value to put in the config. So you created two config files ``config.json`` and ``secret_config.json``. You could check in the ``config.json`` to the Git so everyone can see it. But keep the ``secret_config.json`` private, and only give access to people really need it.

In [9]:
rprint(path_config.read_text())

# content of config.json
{
    # this config file support comments, you can put documentation in the config file
    "_shared": {
        "*.project_name": "my_project"
    },
    "dev": {
        "username": "dev.user"
    },
    "int": {
        "username": "int.user"
    },
    "prod": {
        "username": "prod.user"
    }
}

In [10]:
rprint(path_secret_config.read_text())

# content of secret_config.json
{
    # this config file support comments, you can put documentation in the config file
    "_shared": {
    },
    "dev": {
        "password": "dev.password"
    },
    "int": {
        "password": "int.password"
    },
    "prod": {
        "password": "prod.password"
    }
}

Then you can initialize the config object. You can load the config data from:

- local config json file
- aws parameter store
- aws s3

Since we haven't deployed it yet, so we can only load it from local config file.

In [11]:
# -*- coding: utf-8 -*-

import os
from pathlib import Path
from boto_session_manager import BotoSesManager
from rich import print as rprint

# Read config from local file
dir_here = Path(os.getcwd()).absolute()
path_config = dir_here.joinpath("config.json")
path_secret_config = dir_here.joinpath("secret_config.json")

config = Config.read(
    env_class=Env,
    env_enum_class=EnvEnum,
    path_config=path_config,
    path_secret_config=path_secret_config,
)
rprint(config)

Config(
    data={
        '_shared': {'*.project_name': 'my_project'},
        'dev': {'username': 'dev.user'},
        'int': {'username': 'int.user'},
        'prod': {'username': 'prod.user'}
    },
    secret_data={
        '_shared': {},
        'dev': {'password': 'dev.password'},
        'int': {'password': 'int.password'},
        'prod': {'password': 'prod.password'}
    },
    Env=<class '__main__.Env'>,
    EnvEnum=<enum 'EnvEnum'>,
    _applied_data={
        'dev': {'username': 'dev.user', 'project_name': 'my_project'},
        'int': {'username': 'int.user', 'project_name': 'my_project'},
        'prod': {'username': 'prod.user', 'project_name': 'my_project'}
    },
    _applied_secret_data={
        'dev': {'password': 'dev.password'},
        'int': {'password': 'int.password'},
        'prod': {'password': 'prod.password'}
    },
    _merged={
        'dev': {'username': 'dev.user', 'project_name': 'my_project', 'password': 'dev.password'},
        'int': {'username': 'int.user', 'project_name': 'my_project', 'password': 'int.password'},
        'prod': {'username': 'prod.user', 'project_name': 'my_project', 'password': 'prod.password'}
    }
)

Please note that the ``_applied_data`` is where the shared values are applied to ``data``, and ``_applied_secret_data`` is where the shared values are applied to ``secret_data``. The ``_merged`` is the merged version of ``_applied_data`` and ``_applied_secret_data``.

### Deploy to Config Storage

Now, we can deploy the configuration to the storage. It creates an all-in-one deployment containing all environment data as a backup for disaster recovery or compliance requirements. Additionally, it creates individual per-environment deployments, ensuring that each environment can only access its specific configuration data and cannot access the configuration of other environments.

**Deploy to AWS Parameter Store**

You could set the ``parameter_with_encryption=True or False`` to tell that you want to deploy to AWS Parameter Store.


In [12]:
# Deploy config to AWS Parameter Store
bsm = BotoSesManager(profile_name="bmt_app_dev_us_east_1")

deployment_list = config.deploy(
    bsm=bsm,
    parameter_with_encryption=True,
    verbose=True,
)

+----- ⏱ 🟢 Start 'deploy config to SSM parameter' -----------------------------+
| 
| 🚀️ deploy SSM Parameter 'my_project' ...
| preview at: https://us-east-1.console.aws.amazon.com/systems-manager/parameters/my_project/description?region=us-east-1&tab=Table
| successfully deployed version 1
| 
+----- ⏰ 🟢 End 'deploy config to SSM parameter', elapsed = 0.24 sec -----------+
+----- ⏱ 🟢 Start 'deploy config to SSM parameter' -----------------------------+
| 
| 🚀️ deploy SSM Parameter 'my_project-dev' ...
| preview at: https://us-east-1.console.aws.amazon.com/systems-manager/parameters/my_project-dev/description?region=us-east-1&tab=Table
| successfully deployed version 1
| 
+----- ⏰ 🟢 End 'deploy config to SSM parameter', elapsed = 0.12 sec -----------+
+----- ⏱ 🟢 Start 'deploy config to SSM parameter' -----------------------------+
| 
| 🚀️ deploy SSM Parameter 'my_project-int' ...
| preview at: https://us-east-1.console.aws.amazon.com/systems-manager/parameters/my_project-int/descriptio

In [13]:
rprint(deployment_list)

[
    ConfigDeployment(
        parameter_name='my_project',
        parameter_data={
            'data': {
                '_shared': {'*.project_name': 'my_project'},
                'dev': {'username': 'dev.user'},
                'int': {'username': 'int.user'},
                'prod': {'username': 'prod.user'}
            },
            'secret_data': {
                '_shared': {},
                'dev': {'password': 'dev.password'},
                'int': {'password': 'int.password'},
                'prod': {'password': 'prod.password'}
            }
        },
        project_name='my_project',
        env_name='all',
        deployment=Parameter(
            Name='my_project',
            Value='{"data": {"_shared": {"*.project_name": "my_project"}, "dev": {"username": "dev.user"}, "int": 
{"username": "int.user"}, "prod": {"username": "prod.user"}}, "secret_data": {"_shared": {}, "dev": {"password": 
"dev.password"}, "int": {"password": "int.password"}, "prod": {"password": "prod.password"}}}',
            Type='SecureString',
            Version=1,
            LastModifiedDate=datetime.datetime(2023, 5, 17, 15, 36, 49),
            DataType='text',
            ARN=None,
            Selector=None,
            SourceResult=None,
            Tags={},
            Labels=[]
        ),
        deletion=None
    ),
    ConfigDeployment(
        parameter_name='my_project-dev',
        parameter_data={
            'data': {'_shared': {'*.project_name': 'my_project'}, 'dev': {'username': 'dev.user'}},
            'secret_data': {'_shared': {}, 'dev': {'password': 'dev.password'}}
        },
        project_name='my_project',
        env_name='dev',
        deployment=Parameter(
            Name='my_project-dev',
            Value='{"data": {"_shared": {"*.project_name": "my_project"}, "dev": {"username": "dev.user"}}, 
"secret_data": {"_shared": {}, "dev": {"password": "dev.password"}}}',
            Type='SecureString',
            Version=1,
            LastModifiedDate=datetime.datetime(2023, 5, 17, 15, 36, 50),
            DataType='text',
            ARN=None,
            Selector=None,
            SourceResult=None,
            Tags={},
            Labels=[]
        ),
        deletion=None
    ),
    ConfigDeployment(
        parameter_name='my_project-int',
        parameter_data={
            'data': {'_shared': {'*.project_name': 'my_project'}, 'int': {'username': 'int.user'}},
            'secret_data': {'_shared': {}, 'int': {'password': 'int.password'}}
        },
        project_name='my_project',
        env_name='int',
        deployment=Parameter(
            Name='my_project-int',
            Value='{"data": {"_shared": {"*.project_name": "my_project"}, "int": {"username": "int.user"}}, 
"secret_data": {"_shared": {}, "int": {"password": "int.password"}}}',
            Type='SecureString',
            Version=1,
            LastModifiedDate=datetime.datetime(2023, 5, 17, 15, 36, 50),
            DataType='text',
            ARN=None,
            Selector=None,
            SourceResult=None,
            Tags={},
            Labels=[]
        ),
        deletion=None
    ),
    ConfigDeployment(
        parameter_name='my_project-prod',
        parameter_data={
            'data': {'_shared': {'*.project_name': 'my_project'}, 'prod': {'username': 'prod.user'}},
            'secret_data': {'_shared': {}, 'prod': {'password': 'prod.password'}}
        },
        project_name='my_project',
        env_name='prod',
        deployment=Parameter(
            Name='my_project-prod',
            Value='{"data": {"_shared": {"*.project_name": "my_project"}, "prod": {"username": "prod.user"}}, 
"secret_data": {"_shared": {}, "prod": {"password": "prod.password"}}}',
            Type='SecureString',
            Version=1,
            LastModifiedDate=datetime.datetime(2023, 5, 17, 15, 36, 50),
            DataType='text',
            ARN=None,
            Selector=None,
            SourceResult=None,
            Tags=

**Deploy to AWS S3**

You could set the ``s3dir_config="s3://bucket/..."`` to tell that you want to deploy to AWS S3.

In [14]:
# Deploy config to AWS S3 Store
s3dir_config = f"s3://{bsm.aws_account_id}-us-east-1-artifacts/projects/config_pattern/patterns/multi_env_json/"
deployment_list = config.deploy(
    bsm=bsm,
    s3dir_config=s3dir_config,
)

+----- ⏱ 🟢 Start 'deploy config file to S3' -----------------------------------+
| 
| 🚀️ deploy config file s3://878625312159-us-east-1-artifacts/projects/config_pattern/patterns/multi_env_json/my_project.json ...
| preview at: https://us-east-1.console.aws.amazon.com/s3/object/878625312159-us-east-1-artifacts?prefix=projects/config_pattern/patterns/multi_env_json/my_project.json
| done!
| 
+----- ⏰ 🟢 End 'deploy config file to S3', elapsed = 0.21 sec -----------------+
+----- ⏱ 🟢 Start 'deploy config file to S3' -----------------------------------+
| 
| 🚀️ deploy config file s3://878625312159-us-east-1-artifacts/projects/config_pattern/patterns/multi_env_json/my_project-dev.json ...
| preview at: https://us-east-1.console.aws.amazon.com/s3/object/878625312159-us-east-1-artifacts?prefix=projects/config_pattern/patterns/multi_env_json/my_project-dev.json
| done!
| 
+----- ⏰ 🟢 End 'deploy config file to S3', elapsed = 0.08 sec -----------------+
+----- ⏱ 🟢 Start 'deploy config file to S3

In [15]:
rprint(deployment_list)

[
    ConfigDeployment(
        parameter_name='my_project',
        parameter_data={
            'data': {
                '_shared': {'*.project_name': 'my_project'},
                'dev': {'username': 'dev.user'},
                'int': {'username': 'int.user'},
                'prod': {'username': 'prod.user'}
            },
            'secret_data': {
                '_shared': {},
                'dev': {'password': 'dev.password'},
                'int': {'password': 'int.password'},
                'prod': {'password': 'prod.password'}
            }
        },
        project_name='my_project',
        env_name='all',
        deployment=S3Object(
            expiration=None,
            etag='"e851a25b45b9606ddd65fef7105b64ba"',
            checksum_crc32=None,
            checksum_crc32c=None,
            checksum_sha1=None,
            checksum_sha256=None,
            server_side_encryption='AES256',
            version_id=None,
            sse_customer_algorithm=None,
            sse_customer_key_md5=None,
            see_kms_key_id=None,
            sse_kms_encryption_context=None,
            bucket_key_enabled=None,
            request_charged=None
        ),
        deletion=None
    ),
    ConfigDeployment(
        parameter_name='my_project-dev',
        parameter_data={
            'data': {'_shared': {'*.project_name': 'my_project'}, 'dev': {'username': 'dev.user'}},
            'secret_data': {'_shared': {}, 'dev': {'password': 'dev.password'}}
        },
        project_name='my_project',
        env_name='dev',
        deployment=S3Object(
            expiration=None,
            etag='"ac411b52f2670c32151cb4100dd224a7"',
            checksum_crc32=None,
            checksum_crc32c=None,
            checksum_sha1=None,
            checksum_sha256=None,
            server_side_encryption='AES256',
            version_id=None,
            sse_customer_algorithm=None,
            sse_customer_key_md5=None,
            see_kms_key_id=None,
            sse_kms_encryption_context=None,
            bucket_key_enabled=None,
            request_charged=None
        ),
        deletion=None
    ),
    ConfigDeployment(
        parameter_name='my_project-int',
        parameter_data={
            'data': {'_shared': {'*.project_name': 'my_project'}, 'int': {'username': 'int.user'}},
            'secret_data': {'_shared': {}, 'int': {'password': 'int.password'}}
        },
        project_name='my_project',
        env_name='int',
        deployment=S3Object(
            expiration=None,
            etag='"7163f20371e6da6c6b4e28b807339cf5"',
            checksum_crc32=None,
            checksum_crc32c=None,
            checksum_sha1=None,
            checksum_sha256=None,
            server_side_encryption='AES256',
            version_id=None,
            sse_customer_algorithm=None,
            sse_customer_key_md5=None,
            see_kms_key_id=None,
            sse_kms_encryption_context=None,
            bucket_key_enabled=None,
            request_charged=None
        ),
        deletion=None
    ),
    ConfigDeployment(
        parameter_name='my_project-prod',
        parameter_data={
            'data': {'_shared': {'*.project_name': 'my_project'}, 'prod': {'username': 'prod.user'}},
            'secret_data': {'_shared': {}, 'prod': {'password': 'prod.password'}}
        },
        project_name='my_project',
        env_name='prod',
        deployment=S3Object(
            expiration=None,
            etag='"9f709ac7c27b0912a37d368db37cd96c"',
            checksum_crc32=None,
            checksum_crc32c=None,
            checksum_sha1=None,
            checksum_sha256=None,
            server_side_encryption='AES256',
            version_id=None,
            sse_customer_algorithm=None,
            sse_customer_key_md5=None,
            see_kms_key_id=None,
            sse_kms_encryption_context=None,
            bucket_key_enabled=None,
            request_charged=None
        ),
        deletion=None

### Access Your Config Values in Application Code

In your application code, you could create the config object by reading the config storage. Then use the Python config object to access those config values.

In [16]:
from rich import print as rprint
from boto_session_manager import BotoSesManager

# create boto session manager object for AWS SDK authentication
bsm = BotoSesManager(profile_name="bmt_app_dev_us_east_1")
parameter_name = "my_project-dev"
s3dir_config = f"s3://{bsm.aws_account_id}-us-east-1-artifacts/projects/config_pattern/patterns/multi_env_json/"

You could read the config from AWS Parameter Store. For security reason, assuming that you want to load the "dev" config, you won't be able to access any of the "prod" config from your application runtime.

In [17]:
config = Config.read(
    env_class=Env,
    env_enum_class=EnvEnum,
    bsm=bsm,
    parameter_name=parameter_name,
    parameter_with_encryption=True,
)
rprint(config)
rprint(config.dev)
rprint(f"config.dev.username = {config.dev.username!r}")
rprint(f"config.dev.password = {config.dev.password!r}")

Config(
    data={'_shared': {'*.project_name': 'my_project'}, 'dev': {'username': 'dev.user'}},
    secret_data={'_shared': {}, 'dev': {'password': 'dev.password'}},
    Env=<class '__main__.Env'>,
    EnvEnum=<enum 'EnvEnum'>,
    _applied_data={'dev': {'username': 'dev.user', 'project_name': 'my_project'}},
    _applied_secret_data={'dev': {'password': 'dev.password'}},
    _merged={'dev': {'username': 'dev.user', 'project_name': 'my_project', 'password': 'dev.password'}}
)

Env(project_name='my_project', env_name='dev', username='dev.user', password='dev.password')

config.dev.username = 'dev.user'

config.dev.password = 'dev.password'

In [18]:
# You can NOT access prod config from dev environment
config.prod

KeyError: 'prod'

You could also read the config from AWS S3.

In [19]:
config = Config.read(
    env_class=Env,
    env_enum_class=EnvEnum,
    bsm=bsm,
    s3path_config=f"{s3dir_config}my_project-prod.json",
)
rprint(config)
rprint(config.prod)
rprint(f"config.prod.username = {config.prod.username!r}")
rprint(f"config.prod.password = {config.prod.password!r}")

Config(
    data={'_shared': {'*.project_name': 'my_project'}, 'prod': {'username': 'prod.user'}},
    secret_data={'_shared': {}, 'prod': {'password': 'prod.password'}},
    Env=<class '__main__.Env'>,
    EnvEnum=<enum 'EnvEnum'>,
    _applied_data={'prod': {'username': 'prod.user', 'project_name': 'my_project'}},
    _applied_secret_data={'prod': {'password': 'prod.password'}},
    _merged={'prod': {'username': 'prod.user', 'project_name': 'my_project', 'password': 'prod.password'}}
)

Env(project_name='my_project', env_name='prod', username='prod.user', password='prod.password')

config.prod.username = 'prod.user'

config.prod.password = 'prod.password'

### Deploy Per-Environment Config to Different AWS Accounts

Sometimes, you want to deploy the per-environment config to different AWS accounts. However, the ``Config.deploy(bsm=..., ...)`` API only takes one boto3 session. You could use the ``Config.prepare_deploy()`` API to generate a list of ``ConfigDeployment`` object that represents the all-in-one config deployment and per-environment config deployments. Then 
use the ``ConfigDeployment.deploy_to_ssm_parameter(bsm=..., ...)`` or ``ConfigDeployment.deploy_to_s3(bsm=..., ...)`` method to deploy them to different AWS accounts.


In [20]:
config = Config.read(
    env_class=Env,
    env_enum_class=EnvEnum,
    path_config=path_config,
    path_secret_config=path_secret_config,
)
deployment_list = config.prepare_deploy()
rprint(deployment_list)

[
    ConfigDeployment(
        parameter_name='my_project',
        parameter_data={
            'data': {
                '_shared': {'*.project_name': 'my_project'},
                'dev': {'username': 'dev.user'},
                'int': {'username': 'int.user'},
                'prod': {'username': 'prod.user'}
            },
            'secret_data': {
                '_shared': {},
                'dev': {'password': 'dev.password'},
                'int': {'password': 'int.password'},
                'prod': {'password': 'prod.password'}
            }
        },
        project_name='my_project',
        env_name='all',
        deployment=None,
        deletion=None
    ),
    ConfigDeployment(
        parameter_name='my_project-dev',
        parameter_data={
            'data': {'_shared': {'*.project_name': 'my_project'}, 'dev': {'username': 'dev.user'}},
            'secret_data': {'_shared': {}, 'dev': {'password': 'dev.password'}}
        },
        project_name='my_project',
        env_name='dev',
        deployment=None,
        deletion=None
    ),
    ConfigDeployment(
        parameter_name='my_project-int',
        parameter_data={
            'data': {'_shared': {'*.project_name': 'my_project'}, 'int': {'username': 'int.user'}},
            'secret_data': {'_shared': {}, 'int': {'password': 'int.password'}}
        },
        project_name='my_project',
        env_name='int',
        deployment=None,
        deletion=None
    ),
    ConfigDeployment(
        parameter_name='my_project-prod',
        parameter_data={
            'data': {'_shared': {'*.project_name': 'my_project'}, 'prod': {'username': 'prod.user'}},
            'secret_data': {'_shared': {}, 'prod': {'password': 'prod.password'}}
        },
        project_name='my_project',
        env_name='prod',
        deployment=None,
        deletion=None
    )
]

In [21]:
_ = deployment_list[0].deploy_to_ssm_parameter(
    bsm=bsm, # you can use a different boto3 session to deploy to another AWS account
    parameter_with_encryption=True,
)

+----- ⏱ 🟢 Start 'deploy config to SSM parameter' -----------------------------+
| 
| 🚀️ deploy SSM Parameter 'my_project' ...
| preview at: https://us-east-1.console.aws.amazon.com/systems-manager/parameters/my_project/description?region=us-east-1&tab=Table
| parameter data is the same as existing one, do nothing.
| 
+----- ⏰ 🟢 End 'deploy config to SSM parameter', elapsed = 0.19 sec -----------+


In [22]:
_ = deployment_list[0].deploy_to_s3(
    bsm=bsm, # you can use a different boto3 session to deploy to another AWS account
    s3dir_config=s3dir_config, # you can use another S3 bucket on different AWS account
)

+----- ⏱ 🟢 Start 'deploy config file to S3' -----------------------------------+
| 
| 🚀️ deploy config file s3://878625312159-us-east-1-artifacts/projects/config_pattern/patterns/multi_env_json/my_project.json ...
| preview at: https://us-east-1.console.aws.amazon.com/s3/object/878625312159-us-east-1-artifacts?prefix=projects/config_pattern/patterns/multi_env_json/my_project.json
| config data is the same as existing one, do nothing.
| 
+----- ⏰ 🟢 End 'deploy config file to S3', elapsed = 0.04 sec -----------------+


### Delete Config from Config Storage

At the end, you can delete all config from config storage to save cost. To delete a config, you have to initialize the config object first, otherwise it won't know the parameter name and the list of environment to delete from. Usually, you could use the non-sensitive config json file to do so.

In [23]:
config = Config.read(
    env_class=Env,
    env_enum_class=EnvEnum,
    path_config=path_config,
    path_secret_config=path_secret_config,
)

In [24]:
deployment_list = config.delete(
    bsm=bsm,
    use_parameter_store=True,
)

+----- ⏱ 🟢 Start 'delete config from SSM parameter' ---------------------------+
| 
| 🗑️ delete SSM Parameter 'my_project' ...
| verify at: https://us-east-1.console.aws.amazon.com/systems-manager/parameters/my_project/description?region=us-east-1&tab=Table
| done!
| 
+----- ⏰ 🟢 End 'delete config from SSM parameter', elapsed = 0.10 sec ---------+
+----- ⏱ 🟢 Start 'delete config from SSM parameter' ---------------------------+
| 
| 🗑️ delete SSM Parameter 'my_project-dev' ...
| verify at: https://us-east-1.console.aws.amazon.com/systems-manager/parameters/my_project-dev/description?region=us-east-1&tab=Table
| done!
| 
+----- ⏰ 🟢 End 'delete config from SSM parameter', elapsed = 0.09 sec ---------+
+----- ⏱ 🟢 Start 'delete config from SSM parameter' ---------------------------+
| 
| 🗑️ delete SSM Parameter 'my_project-int' ...
| verify at: https://us-east-1.console.aws.amazon.com/systems-manager/parameters/my_project-int/description?region=us-east-1&tab=Table
| done!
| 
+----- ⏰ 🟢 End 

In [25]:
rprint(deployment_list)

[
    ConfigDeployment(
        parameter_name='my_project',
        parameter_data={
            'data': {
                '_shared': {'*.project_name': 'my_project'},
                'dev': {'username': 'dev.user'},
                'int': {'username': 'int.user'},
                'prod': {'username': 'prod.user'}
            },
            'secret_data': {
                '_shared': {},
                'dev': {'password': 'dev.password'},
                'int': {'password': 'int.password'},
                'prod': {'password': 'prod.password'}
            }
        },
        project_name='my_project',
        env_name='all',
        deployment=None,
        deletion=True
    ),
    ConfigDeployment(
        parameter_name='my_project-dev',
        parameter_data={
            'data': {'_shared': {'*.project_name': 'my_project'}, 'dev': {'username': 'dev.user'}},
            'secret_data': {'_shared': {}, 'dev': {'password': 'dev.password'}}
        },
        project_name='my_project',
        env_name='dev',
        deployment=None,
        deletion=True
    ),
    ConfigDeployment(
        parameter_name='my_project-int',
        parameter_data={
            'data': {'_shared': {'*.project_name': 'my_project'}, 'int': {'username': 'int.user'}},
            'secret_data': {'_shared': {}, 'int': {'password': 'int.password'}}
        },
        project_name='my_project',
        env_name='int',
        deployment=None,
        deletion=True
    ),
    ConfigDeployment(
        parameter_name='my_project-prod',
        parameter_data={
            'data': {'_shared': {'*.project_name': 'my_project'}, 'prod': {'username': 'prod.user'}},
            'secret_data': {'_shared': {}, 'prod': {'password': 'prod.password'}}
        },
        project_name='my_project',
        env_name='prod',
        deployment=None,
        deletion=True
    )
]

In [26]:
deployment_list = config.delete(
    bsm=bsm,
    s3dir_config=s3dir_config,
)

+----- ⏱ 🟢 Start 'delete config file from S3' ---------------------------------+
| 
| 🗑️ delete config file s3://878625312159-us-east-1-artifacts/projects/config_pattern/patterns/multi_env_json/my_project.json ...
| preview at: https://us-east-1.console.aws.amazon.com/s3/object/878625312159-us-east-1-artifacts?prefix=projects/config_pattern/patterns/multi_env_json/my_project.json
| done!
| 
+----- ⏰ 🟢 End 'delete config file from S3', elapsed = 0.08 sec ---------------+
+----- ⏱ 🟢 Start 'delete config file from S3' ---------------------------------+
| 
| 🗑️ delete config file s3://878625312159-us-east-1-artifacts/projects/config_pattern/patterns/multi_env_json/my_project-dev.json ...
| preview at: https://us-east-1.console.aws.amazon.com/s3/object/878625312159-us-east-1-artifacts?prefix=projects/config_pattern/patterns/multi_env_json/my_project-dev.json
| done!
| 
+----- ⏰ 🟢 End 'delete config file from S3', elapsed = 0.08 sec ---------------+
+----- ⏱ 🟢 Start 'delete config file from 

In [27]:
rprint(deployment_list)

[
    ConfigDeployment(
        parameter_name='my_project',
        parameter_data={
            'data': {
                '_shared': {'*.project_name': 'my_project'},
                'dev': {'username': 'dev.user'},
                'int': {'username': 'int.user'},
                'prod': {'username': 'prod.user'}
            },
            'secret_data': {
                '_shared': {},
                'dev': {'password': 'dev.password'},
                'int': {'password': 'int.password'},
                'prod': {'password': 'prod.password'}
            }
        },
        project_name='my_project',
        env_name='all',
        deployment=None,
        deletion=True
    ),
    ConfigDeployment(
        parameter_name='my_project-dev',
        parameter_data={
            'data': {'_shared': {'*.project_name': 'my_project'}, 'dev': {'username': 'dev.user'}},
            'secret_data': {'_shared': {}, 'dev': {'password': 'dev.password'}}
        },
        project_name='my_project',
        env_name='dev',
        deployment=None,
        deletion=True
    ),
    ConfigDeployment(
        parameter_name='my_project-int',
        parameter_data={
            'data': {'_shared': {'*.project_name': 'my_project'}, 'int': {'username': 'int.user'}},
            'secret_data': {'_shared': {}, 'int': {'password': 'int.password'}}
        },
        project_name='my_project',
        env_name='int',
        deployment=None,
        deletion=True
    ),
    ConfigDeployment(
        parameter_name='my_project-prod',
        parameter_data={
            'data': {'_shared': {'*.project_name': 'my_project'}, 'prod': {'username': 'prod.user'}},
            'secret_data': {'_shared': {}, 'prod': {'password': 'prod.password'}}
        },
        project_name='my_project',
        env_name='prod',
        deployment=None,
        deletion=True
    )
]